# Using rapidfuzz to extract affiliatuons from RES

First the JSON output file from tezseract is read in and made into a useable dataframe

In [2]:
#Importing the libraries
import json
import pandas as pd
from rapidfuzz import process
import requests

# Import JSON file from Github repository
tesseract_input = requests.get('https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1_Input%20files%20and%20processing%20input%20files/1_1_Input%20files/Tesseract%20affiliation%20output/RES/RES_aff_output_1940_2010.json')
json_data = tesseract_input.json()

# Import JSON file
#f = open('/Users/MattJohnson/Desktop/Affiliations/1_Input files and processing input files/1_1_Input files/Tesseract affiliation output/RES/RES_aff_output_1940_2010.json')
#json_data = json.load(f)

# Define Journal and Dataframe
Journal_Name = 'RES'
Reference_Data_List = []

# Extract Json Data into dataframe
for key1, value1 in json_data.items():
    Article_ID = key1
    content_type = value1["content_type"]
    author_list = value1["authors"]
    authorsNo = len(author_list)
    stable_url = value1["stable_url"]
    for key2 in value1["affiliations"]:
        Ref_type = key2
        if key2 == "found":
            ref_string = value1["affiliations"][key2]
        else: 
            for key3 in value1["affiliations"]["raw"]:
                #if len(value1["affiliations"]["raw"]) > 1:
                    #print(key1, "   ",len(value1["affiliations"]["raw"]))
                if key3 == "1":
                    print(key3)
                    ref_string = value1["affiliations"][key2][key3]
                    print(ref_string)

    Reference_Data_List.append({'Journal_Name' : Journal_Name,
    'Authors_count' : authorsNo, 
    'Authors_list' : author_list,
    'Article_ID' : Article_ID, 'content_type' : content_type,
    'stable_url' : stable_url, 
    'Ref_type': Ref_type,  'ref_string':ref_string })

# Create a dataframe from the extracted data
RES_aff_ref_output_1940_2010 = pd.DataFrame.from_records(Reference_Data_List) 

#Basic view of data
print(RES_aff_ref_output_1940_2010.head())
print(RES_aff_ref_output_1940_2010.shape)

#Use line below to send data to excel to view full file
#RES_aff_ref_output_1940_2010.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/Working files/RES_aff_ref_output_1940_2010.xlsx')        

1
['Two Liquidity Functions and the Rate\nof Intetest: |. A Simple Dynamic Model\n\nThis paper contains a model describing the determination of the rate of interest.\nThe model is in principle founded on the Keynesian theory of the interrelation between\nthe quantity of money and the rate of interest. When using this theory it is important\nto distinguish between different kinds of \'\'interest rates" and different kinds of\n" money." The model is concerned with the latter distinction, which is stressed by\nintroducing two sectors—\'\' the banks" and \'\'the public "—whose dispositions on\nthe credit market will be treated separately. In consequence two \'\' quantities of\nmoney "\' and two liquidity functions are obtained. The importance of this distinction\nis further stressed by introducing the banks\' reserve ratio of cash to the public\'s\ndeposits as a link between the two liquidity functions. The model operates exclusively\nin terms of these two functions and thus the analysis o

Next the Unique list of universities is read in, this is used as a database to check the affiliton string against using rapidfuzz. 

In [3]:
#testdata = RES_1940_2010.loc[0:20].reset_index(drop = True)
testdata = RES_aff_ref_output_1940_2010

for i in range(len(testdata)):
    if testdata.iloc[i]['Ref_type'] == "raw":
        testdata.loc[i,['ref_string_clean1']] =  " ".join(testdata.iloc[i]['ref_string'])
        #x = " ".join(testdata.iloc[i]['ref_string'])
        #print(x)
        testdata.loc[i,['ref_string_clean1']] = testdata.iloc[i]['ref_string_clean1'].strip().title()
    else:
        testdata.loc[i,['ref_string_clean1']] = testdata.iloc[i]['ref_string'].strip().title()

print(testdata.columns.values)

['Journal_Name' 'Authors_count' 'Authors_list' 'Article_ID' 'content_type'
 'stable_url' 'Ref_type' 'ref_string' 'ref_string_clean1']


In [5]:
#Reading in the master file and correctly formatting the article ID to match back onto the meltef file

#Load in master from Github
RES_master = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1_Input%20files%20and%20processing%20input%20files/1_1_Input%20files/Master%20lists/Processed/RES_processed.xlsx")

#Local store of RES_master
#RES_master = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/1_Input files and processing input files/1_1_Input files/Master lists/Processed/RES_processed.xlsx')

for i in range(len(RES_master)):
    RES_master.loc[i,['Article_ID']] = RES_master.loc[i,['stable_url']].to_string().rsplit('/',1)[-1]

RES_master_condenced  = RES_master[['Article_ID','year']]
RES_master_condenced['year'] = RES_master_condenced['year'].astype('int')
RES_master_condenced.head()

/var/folders/23/ft9qf9c11rj_vqhbjml910f40000gn/T/ipykernel_30436/3139316167.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RES_master_condenced['year'] = RES_master_condenced['year'].astype('int')


,Article_ID,year
0,26543955,2018
1,26543956,2018
2,26543957,2018
3,26543958,2018
4,26543959,2018


In [6]:
# Merging the masterfile onto the new melted file to get additional data
testdata_merged = testdata.merge(RES_master_condenced, how = 'left', on = 'Article_ID')
testdata_merged = testdata_merged.sort_values(by=['year'])
testdata = testdata_merged.reset_index(drop = True)

In [7]:
# Excel output to get an overview of the data
#testdata.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/RES/Test files/RES_aff_ref_output_1940_2010_affs_out setup.xlsx') 
print(testdata.head())

  Journal_Name  Authors_count        Authors_list Article_ID content_type  \
0          RES              1       [Oscar Lange]    2967419      Article   
1          RES              1        [M. Polanyi]    2967413      Article   
2          RES              1         [L. Rostas]    2967414      Article   
3          RES              1     [Marian Bowley]    2967415      Article   
4          RES              1  [G. L. S. Shackle]    2967416      Article   

                             stable_url Ref_type  \
0  https://www.jstor.org/stable/2967419    found   
1  https://www.jstor.org/stable/2967413    found   
2  https://www.jstor.org/stable/2967414    found   
3  https://www.jstor.org/stable/2967415    found   
4  https://www.jstor.org/stable/2967416    found   

                                          ref_string  \
0  COMPLEMENTARITY AND INTERRELATIONS OF SHIFTS 6...   
1  ECONOMICS BY MOTION SYMBOLS I9\n\nfrom the rep...   
2  CAPITAL LEVIES IN CENTRAL EUROPE, r91i9-1924 3...   


In [8]:
import re
from rapidfuzz import fuzz
import math

for x in range(len(testdata)):
#for x in range(2068,2072):
#for x in range(3627,2063):
    authors = testdata.iloc[x]['Authors_list']
    for m in range(len(authors)):
        authors[m] = authors[m].title()
        #print(authors[m])
    str1 = testdata.iloc[x]['ref_string_clean1']

#Splitting up the string into authors lines
    ref_split = re.split(r"\n",str1 )
    #Removing blank lines from the split
    while("" in ref_split):
        ref_split.remove("")

#Finding the authors name and corrosponding affiliations
    count_found = 0
    #Loop through author names in author list
    for d in range(len(authors)):
        #print(authors[d])

        #Loop through the data that is split into possible authors affiliations
        for j in range(len(ref_split)):
            #Check if the author name is in the split string
            ratio_author  = fuzz.partial_ratio(ref_split[j], authors[d])
            
            #print(ratio,x)
            if ratio_author > 93:
                count_found = count_found +1
                #print(count_found)

                #append the next line to a column for each author
                # This will ensure each author is accounted for
                #print(testdata.iloc[x]['year'])
                if(testdata.iloc[x]['year']) < 1971:
                    remove_author = re.sub(authors[d], "", ref_split[j])  
                    testdata.loc[x, str(d)+'_author_affs'] = remove_author
                else:
                    if j+1 < len(ref_split):
                        #print(ratio_author)
                        #print(ref_split[j+1], authors[d])
                        testdata.loc[x, str(d)+'_author'] = authors[d]

                        # checking for short erroenous splits
                        if len(ref_split[j+1]) > 3:
                            testdata.loc[x, str(d)+'_author_affs'] = ref_split[j+1]
                        else:
                            testdata.loc[x, str(d)+'_author_affs'] = ''
                        #print(testdata.loc[x, str(d)+'_author_affs'])
                print(testdata.loc[x, str(d)+'_author_affs'])

            # Prevents double finding of author name, sometimes name occurs twice 
            # but first occurance is the affiliation
            if count_found == len(authors):
                break
                #else:


    print(x)

    

Chicago. .
0
1
2
3
London. G. L. S. Shacele.
4
Ames, Iowa, U.S.A. .
5
.
6
7
P. Sargant Florence.
8
9
10
11
London. .
12
13
London. .
14
The Hague, Holland. .
15
16
Columbus, Ohio. Beénjamin Caplan.
17
Cambridge, Mass. .
18
London..
19
Chicago, Illinois. .
20
 
 
21
22
Cambridge. .
23
24
 
25
26
Glasgow. E. M. Chossudowsskky.
27
 
28
29
Manchester. .
30
31
Ames, Iowa, U.S.A. .
32
Chicago, Illinois, U.S.A. .
33
Edinburgh. .
34
Manchester. .
35
Cambridge. .
36
Presibury. Unsula K. Hicks.
37
London. .
38
.
D. G. Champernowne.
39
London. .
40
41
Glasgow. .
42
Oxford. J. L. Nichorson.
43
Ames, Iowa. .
44
London. .
45
Manchester. .
46
47
48
 
49
50
51
52
London. k.
53
Manchester. .
54
Manchester .
55
56
Manchester. .
57
58
59
60
61
62
Cambridge. .
63
.*
Birmingham. P. Sangant Florence.
64
London. .
65
Monireal. .
66
67
68
Rennes (France). .
69
70
Washington, D.C. Tibor Scrtovszky.
71
72
73
74
Manchester. .
75
Cambridge. ,
76
.!
Birmingham..
77
London. .
78
Washington, D.C. .
79
Oxford. .
80
P

In [9]:
# Creating a dictionary for consolidation
# Read in from Excel sheet, 2 columns first one with incorrect spelling and second one with correct spellig

cities = pd.read_excel('https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/2_Extracting%20affilitations/3_consolidation/Files/Consolidated_list_Final.xlsx', 'Cities')
#cities = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Cities')
cities = cities[['Incorrect spelling', 'Correct spelling']]

misspellings = pd.read_excel('https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/2_Extracting%20affilitations/3_consolidation/Files/Consolidated_list_Final.xlsx', 'Misspellings')
#misspellings = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Misspellings')
misspellings = misspellings[['Incorrect spelling', 'Correct spelling']]

comma_in_name = pd.read_excel('https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/2_Extracting%20affilitations/3_consolidation/Files/Consolidated_list_Final.xlsx', 'Comma in name')
#comma_in_name = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Comma in name')
comma_in_name = comma_in_name[['Incorrect spelling', 'Correct spelling']]

colleges_full = pd.read_excel('https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/2_Extracting%20affilitations/3_consolidation/Files/Consolidated_list_Final.xlsx', 'Colleges Full')
#colleges_full = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Colleges Full')
colleges_full = colleges_full[['Incorrect spelling', 'Correct spelling']]

colleges = pd.read_excel('https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/2_Extracting%20affilitations/3_consolidation/Files/Consolidated_list_Final.xlsx', 'Colleges')
#colleges = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Colleges')

correct = pd.read_excel('https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/2_Extracting%20affilitations/3_consolidation/Files/Consolidated_list_Final.xlsx', 'Correct')
#correct = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Correct')

frames = [cities, misspellings, comma_in_name,colleges_full ,colleges,correct]
consolidated_affiliations = pd.concat(frames)
print(consolidated_affiliations.shape)

for i in range(len(consolidated_affiliations)):
    #Replaceing title case and stripping leading space
    #affs.iat[i,0] = affs.iat[i,0].strip().title()
    consolidated_affiliations.iloc[i]['Incorrect spelling'] = consolidated_affiliations.iloc[i]['Incorrect spelling'].strip().title()
    #affs.iat[i,0] = affs.iat[i,0].strip().upper()

consolidated_affiliations_dict = consolidated_affiliations.set_index('Incorrect spelling').to_dict()['Correct spelling']
print(consolidated_affiliations_dict)

(1716, 2)
{'Ames, Iowa': 'Ames Iowa United States (City)', 'Ames, Iowa, Usa': 'Ames Iowa United States (City)', 'Antwerp': 'Antwerp Belguim (City)', 'Auckland, New Zealand': 'Auckland New Zealand (City)', 'Auckland, Nz': 'Auckland New Zealand (City)', 'Baghdad': 'Baghdad Iraq (City)', 'Baltimore': 'Baltimore Maryland (City)', 'Baltimore, Usa Kzlvin J Lancaster': 'Baltimore Maryland (City)', 'Belfast': 'Belfast Ireland (City)', 'Berkeley': 'Berkeley California United States (City)', 'Berkeley, California': 'Berkeley California United States (City)', 'Birmingham': 'Birmingham United Kingdom (City)', 'Birmingham And Ames, Iowa': 'Birmingham United Kingdom (City)', 'Bloomington, Indiana': 'Bloomington Indiana United States (City)', 'Bristol': 'Bristol United Kingdom (City)', 'Brooklyn, New York': 'Brooklyn New York United States (City)', 'Brunswick, Maine Davidp Braybrooke': 'Brunswick Maine United States (City)', 'Brussels': 'Brussels Belgium (City)', 'Brussels, Belgium': 'Brussels Belgiu

In [10]:
import numpy as np
testdata = testdata.replace(np.nan, '', regex=True)
testdata["4_author_affs"] = ""
testdata["5_author_affs"] = ""

testdata["0_author_affs_consolidated"] = ""
testdata["0_author_affs_consolidated_0"] = ""
testdata["0_author_affs_consolidated_1"] = ""
testdata["0_author_affs_consolidated_2"] = ""
testdata["0_author_affs_consolidated_3"] = ""
testdata["0_author_affs_consolidated_4"] = ""

testdata["1_author_affs_consolidated"] = ""
testdata["1_author_affs_consolidated_0"] = ""
testdata["1_author_affs_consolidated_1"] = ""
testdata["1_author_affs_consolidated_2"] = ""
testdata["1_author_affs_consolidated_3"] = ""
testdata["1_author_affs_consolidated_4"] = ""

testdata["2_author_affs_consolidated"] = ""
testdata["2_author_affs_consolidated_0"] = ""
testdata["2_author_affs_consolidated_1"] = ""
testdata["2_author_affs_consolidated_2"] = ""
testdata["2_author_affs_consolidated_3"] = ""
testdata["2_author_affs_consolidated_4"] = ""

testdata["3_author_affs_consolidated"] = ""
testdata["3_author_affs_consolidated_0"] = ""
testdata["3_author_affs_consolidated_1"] = ""
testdata["3_author_affs_consolidated_2"] = ""
testdata["3_author_affs_consolidated_3"] = ""
testdata["3_author_affs_consolidated_4"] = ""

testdata["4_author_affs_consolidated"] = ""
testdata["4_author_affs_consolidated_0"] = ""
testdata["4_author_affs_consolidated_1"] = ""
testdata["4_author_affs_consolidated_2"] = ""
testdata["4_author_affs_consolidated_3"] = ""

testdata["5_author_affs_consolidated"] = ""
testdata["5_author_affs_consolidated_0"] = ""

for i in range(len(testdata)):
#for i in range(0,50):
#for i in range(2113,2114):
    authors = testdata.iloc[i]['Authors_list']
    for auth_no in range(len(authors)):
        #print(testdata.iloc[i]['0_author_affs'])
        testdata.loc[i,str(auth_no)+'_author_affs'] =  re.sub(r'\.','',testdata.iloc[i][str(auth_no)+'_author_affs'])
        testdata.loc[i,str(auth_no)+'_author_affs'] =  re.sub(r'The ','',testdata.iloc[i][str(auth_no)+'_author_affs'])
        #print(testdata.iloc[i]['0_author_affs'])
        list = []

    #Loop through the list of incorrect spellings and replace with corrected spellings
        for key in consolidated_affiliations_dict.keys():
            #testdata.loc[i,'0_author_affs_consolidated']  = testdata.iloc[i]['0_author_affs'].replace(key, consolidated_affiliations_dict[key])
            if key in testdata.iloc[i][str(auth_no)+'_author_affs']:
                list.append(key)
                #print(len(list))
                #print((key))
                #print(consolidated_affiliations_dict[key])

    #If there is more than 1 match in the consolidation it needs to match on the longer one, solves for college issue
                if len(list) <= 1:
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                else:
                    #print(consolidated_affiliations_dict[list[0]])
                    #print(consolidated_affiliations_dict[list[1]])
                    if(len(consolidated_affiliations_dict[list[0]]) == len(consolidated_affiliations_dict[list[1]])):
                        #print("HERE")
                        if len(list[0]) > len(list[1]):
                            #print("HERE")
                            testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(list[0], consolidated_affiliations_dict[list[0]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                            #print(list[0])
                        else:
                            #print("this")
                            first_fix = re.sub(list[1], consolidated_affiliations_dict[list[1]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',') 
                            testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
                    else:
                        #print("else")
                        #print(key)
                        first_fix = re.sub(list[0], consolidated_affiliations_dict[list[0]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',') 
                        #print(first_fix)
                        testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 

        print(i)
        print('Full ',testdata.loc[i,str(auth_no)+'_author_affs_consolidated'])
        #else: 
        #    print("non existant")
        auths_affs_split_And = re.split(' And ', testdata.iloc[i][str(auth_no)+'_author_affs_consolidated'])
        #print(auths_affs_split_And)
        #print(len(auths_affs_split_And))

        for aff in range(len(auths_affs_split_And)):
            testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(aff)]  = auths_affs_split_And[aff].strip().rstrip(',') 
            print('Auth', str(auth_no), str(aff) , testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(aff)])


0
Full  Chicago Illinois United States (City)
Auth 0 0 Chicago Illinois United States (City)
1
Full  
Auth 0 0 
2
Full  
Auth 0 0 
3
Full  
Auth 0 0 
4
Full  London United Kingdom (City)
Auth 0 0 London United Kingdom (City)
5
Full  Ames Iowa United States (City), USA
Auth 0 0 Ames Iowa United States (City), USA
6
Full  
Auth 0 0 
7
Full  
Auth 0 0 
8
Full  
Auth 0 0 
8
Full  
Auth 1 0 
9
Full  
Auth 0 0 
10
Full  
Auth 0 0 
11
Full  
Auth 0 0 
12
Full  London United Kingdom (City)
Auth 0 0 London United Kingdom (City)
13
Full  
Auth 0 0 
14
Full  London United Kingdom (City)
Auth 0 0 London United Kingdom (City)
15
Full  Hague Holland (City)
Auth 0 0 Hague Holland (City)
16
Full  
Auth 0 0 
17
Full  Columbus Ohio United States (CIty)
Auth 0 0 Columbus Ohio United States (CIty)
18
Full  Cambridge Massachusetts United States (City)
Auth 0 0 Cambridge Massachusetts United States (City)
19
Full  
Auth 0 0 
19
Full  London United Kingdom (City)
Auth 1 0 London United Kingdom (City)
20
Full

In [11]:
print(testdata.columns.values)

['Journal_Name' 'Authors_count' 'Authors_list' 'Article_ID' 'content_type'
 'stable_url' 'Ref_type' 'ref_string' 'ref_string_clean1' 'year'
 '0_author_affs' '1_author_affs' '2_author_affs' '3_author_affs'
 '0_author' '1_author' '2_author' '3_author' '4_author_affs'
 '5_author_affs' '0_author_affs_consolidated'
 '0_author_affs_consolidated_0' '0_author_affs_consolidated_1'
 '0_author_affs_consolidated_2' '0_author_affs_consolidated_3'
 '0_author_affs_consolidated_4' '1_author_affs_consolidated'
 '1_author_affs_consolidated_0' '1_author_affs_consolidated_1'
 '1_author_affs_consolidated_2' '1_author_affs_consolidated_3'
 '1_author_affs_consolidated_4' '2_author_affs_consolidated'
 '2_author_affs_consolidated_0' '2_author_affs_consolidated_1'
 '2_author_affs_consolidated_2' '2_author_affs_consolidated_3'
 '2_author_affs_consolidated_4' '3_author_affs_consolidated'
 '3_author_affs_consolidated_0' '3_author_affs_consolidated_1'
 '3_author_affs_consolidated_2' '3_author_affs_consolidated_3'
 

In [12]:
testdata = testdata[['Journal_Name' ,'Authors_count', 'Authors_list', 'Article_ID', 'content_type',
 'stable_url', 'Ref_type' ,'ref_string', 'ref_string_clean1', 'year',
 '0_author_affs', 
 '0_author_affs_consolidated',
 '0_author_affs_consolidated_0',
 '0_author_affs_consolidated_1', 
 '0_author_affs_consolidated_2', 
 '0_author_affs_consolidated_3',
 '1_author_affs', 
 '1_author_affs_consolidated',
 '1_author_affs_consolidated_0',
 '1_author_affs_consolidated_1',
 '2_author_affs',
 '2_author_affs_consolidated',
 '2_author_affs_consolidated_0',
 '2_author_affs_consolidated_1', 
 '2_author_affs_consolidated_2' ,
 '3_author_affs',
 '3_author_affs_consolidated', 
 '3_author_affs_consolidated_0',
 '3_author_affs_consolidated_1',
 '4_author_affs',
 '4_author_affs_consolidated',
 '4_author_affs_consolidated_0',
 '5_author_affs', 
 '5_author_affs_consolidated']]

In [13]:
# Excel output to get an overview of the data
testdata.head
testdata.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/3_consolidation/Files/RES_Affiliations.xlsx') 